# 导入Pandas 包

In [1]:
import pandas as pd

# 预备知识

## 数据结构

每张牌2个维度：一个是数字，一个是花

花色我们用黑桃Spade，梅花Club，方块Diamonds，红桃Hearts用首字母标示S,C,D,H

数字用2，3，4，5，6，7，8，9，T，J，Q，K，A标示

用如下的方式标示手中的五张牌

In [2]:
a='9D 9H 9S 9C AC'
b='2C 7C 8C 9C TC'

c='2S 3S 4S 5S 7D'
d='2S 9S 7D 8S TD'

## python 的一些函数

In [3]:
#split函数，将字符串转化成list
a.split()

['9D', '9H', '9S', '9C', 'AC']

In [4]:
#list中for 语句的小技巧
print([i for i in a.split()])
print([a for a,b in a.split()])
print([b for a,b in a.split()])

['9D', '9H', '9S', '9C', 'AC']
['9', '9', '9', '9', 'A']
['D', 'H', 'S', 'C', 'C']


In [5]:
# 查找字符串中字符位置 index（）
print('--23456789TJQKA'.index('A'))
print('--23456789TJQKA'.index('7'))

14
7


In [6]:
# 语句判断
print(1==2)
print(2==2)
print(2>=2)
print('A'=='A')
print('AA'=='A')

False
True
True
True
False


In [7]:
#计算对象中，有多少个不重复的元素
print(set([1,2,2,3,4]))
print(set(['1','2','2','3','4']))
print(set('1222333'))
print(len(set([1,2,2,3,4])))

{1, 2, 3, 4}
{'4', '1', '3', '2'}
{'1', '3', '2'}
4


In [8]:
#计算对象中，某元素的个数
print([1,2,3,4,5].count(1))
print([1,1,1,2,3].count(1))

1
3


In [9]:
#比较多个对象大小max()按照顺序比较
print(max(9,8))
print(max([9,8,7,6,5],[10,9,8,7,6]))
print(max([10,8,7,6,5],[10,9,8,7,6]))
print(max([10,9,7,6,5],[10,9,8,7,6]))
print(max((9,[9,8,7,6,5]),(9,[10,8,7,6,5])))

9
[10, 9, 8, 7, 6]
[10, 9, 8, 7, 6]
[10, 9, 8, 7, 6]
(9, [10, 8, 7, 6, 5])


# 德扑判断卡牌大小

## 按照数字对五张牌进行排序

In [296]:
def card_ranks(hand):
    ranks = ['--23456789TJQKA'.index(r) for r, s in hand]
    ranks.sort(reverse=True)
    return  [5, 4, 3, 2, 1] if ranks == [14, 5, 4, 3, 2] else ranks

In [297]:
a='9D 9H 9S 9C AC'
print(card_ranks(a.split()))

[14, 9, 9, 9, 9]


## 判断顺子/同花/两对/n张相同牌

In [298]:
#ranks 指的是数字排序后的list，例如[5，4，3，2，1]
#hand 指的是牌，例如['1S','2S','6S','8S','9S']

# 判断顺子 
def straight(ranks):
    return (max(ranks) - min(ranks)) == 4 and len(set(ranks)) == 5 #最大牌减最小牌等于4，且五张牌不重复

#判断同花
def flush(hand):
    suit = [s for r, s in hand]
    return len(set(suit)) == 1 #五张牌的花色相同

#判断n张牌相同
def kind(n, ranks):
    for s in ranks:
        if ranks.count(s) == n : return s #如果某元素在对象中的个数等于我们定义的n，则返回该元素具体数值
    return None

#判断是否有两对
def two_pair(ranks):
    pair = kind(2, ranks)
    lowpair = kind(2, list(reversed(ranks))) #在有两对儿的情况下，因为赋新值会使得原值被覆盖，所以把原来的rank换个方向，就会得到另一对儿
    if pair != lowpair:
        return (pair, lowpair)
    else:
        return None

In [299]:
ranks_test=[9,8,7,5,5]
hand_test=['1S','2S','6S','8S','9S']

In [300]:
straight(ranks_test)

False

In [301]:
flush(hand_test)

True

In [302]:
kind(2,ranks_test)

5

In [303]:
two_pair(ranks_test) #没有输出，是因为没有两对

## 为不同牌型赋予权重

In [304]:
def hand_rank(hand):
    "Return a value indicating the ranking of a hand."
    ranks = card_ranks(hand) 
    if straight(ranks) and flush(hand):                #同花顺
        return (9, max(ranks))                         #返回（权重9，顺子的最大牌面）
    elif kind(4, ranks):                               #炸弹
        return (8, kind(4, ranks), kind(1, ranks))     #返回（权重8，炸弹的牌面，剩余的一张牌的大小）
    elif kind(3, ranks) and kind(2, ranks):            #3带2
        return (7, kind(3, ranks), kind(2, ranks))     #返回（权重7，三张的牌面，两张的牌面）
    elif flush(hand):                                  #同花
        return (6,ranks)                               #返回（权重6，五张牌的从大到小的排序）
    elif straight(ranks):                              #顺子
        return (5, max(ranks))                         #返回（权重5，顺子的最大牌面）
    elif kind(3, ranks):                               #三张
        return (4, kind(3, ranks), ranks)              #返回（权重4，三张的牌面，五张牌的从大到小的排序）
    elif two_pair(ranks):                              #两对
        return (3, two_pair(ranks), ranks)             #返回（权重3，（较大的一对儿的牌面，较小的一对儿的牌面），五张牌的从大到小的顺序）
    elif kind(2, ranks):                               #两张
        return (2, kind(2, ranks), ranks)              #返回（权重2，两张的牌面，五张牌的从大到小的顺序）
    else:                                              #一张
        return (1, ranks)                              #返回（权重1，五张牌的从大到小的顺序）

## 比较大小

In [305]:
def poker(hands):
    return max(hands, key=hand_rank) # 利用max（）是按照顺序比较，所以首先比较权重，然后如果权重一样，则比较下一位

In [306]:
def whowins(hands):
    return(hands.index(poker(hands)))

In [307]:
# create hand universe

In [308]:
fullnumberlist =reversed (['2','3','4','5','6','7','8','9','T','J','Q','K','A'])
fullsuitlist = ['S','C','D','H']

In [309]:
fulldeck=[]

for i in fullnumberlist :
    for y in fullsuitlist :
        fulldeck.append(i+y)
        

In [310]:
handuniverse=[]
len_handuniverse=0
for a in range (1,53) :
    for b in range (a+1,53) :
        for c in range (b+1,53) :
            for d in range (c+1,53):
                for e in range (d+1,53):
                    handuniverse.append([fulldeck[a-1],fulldeck[b-1],fulldeck[c-1],fulldeck[d-1],fulldeck[e-1]])
                    len_handuniverse=len_handuniverse +1
                        

In [311]:
handuniverse_ranked = sorted(handuniverse, key=hand_rank, reverse= True)   #sort the hand universe

In [312]:
# for any 5 cards, what is the winning probability. It needs to be improved to consider the equal-hands situation
def winprob_fivecards (hand) :  
    sorted_hand=card_sort(hand)
    return (1-handuniverse_ranked.index(sorted_hand) / len_handuniverse)

In [365]:
def winprob_twocards (twocards) :
    twocards=['AH','AD']
    twocards_0 = fulldeck.index(twocards[0])
    twocards_1 = fulldeck.index(twocards[1])
    sum_prob=0
    i=0
    for a in range (52) :
        for b in range (a+1,52) :
            for c in range (b+1,52) :
                if twocards_0 not in [a,b,c] and twocards_1 not in [a,b,c] :
                    #print([fulldeck[twocards_0],fulldeck[twocards_1],fulldeck[a],fulldeck[b],fulldeck[c]])
                    sum_prob=sum_prob+winprob_fivecards([fulldeck[twocards_0],fulldeck[twocards_1],fulldeck[a],fulldeck[b],fulldeck[c]])
                    i=i+1
                    
    return [sum_prob/i,i]
    


In [314]:
# sort the hand into the order that can be looked up in the handuniverse
def card_sort(hand):
    fullcardnum='--23456789TJQKA'
    fullsuitlist = ['S','C','D','H']
    ranks_1 = [fullcardnum.index(r) for r, s in hand] 
    ranks_2 = [fullsuitlist.index(s) for r, s in hand]
    card=[]
    for i in range(5) :
        card.append ([ranks_1[i],ranks_2[i]])
    sorted_card_num = sorted (card, key = lambda x:(-x[0],x[1]))
    sorted_card=[]
    for i in range(5):
        sorted_card.append(str(fullcardnum[sorted_card_num[i][0]])+fullsuitlist[sorted_card_num[i][1]])
    return sorted_card


In [344]:

testhand = ['AC', 'AD', 'AH', 'AS', 'KC']
winprob_fivecards(testhand)

0.9999842244590144

In [369]:
winprob_twocards(['3H','JD'])

[0.9273560590641274, 19600]

In [367]:
50*49*48/6


19600.0

# Test

In [72]:
a='9D 9H 9S 9C AC'.split()
b='2C 7C 8C 9C TC'.split()

c='2S 3S 4S 5S 7D'.split()
d='2S 9S 7D 8S TD'.split()

In [73]:
poker([a,b,c,d])

['9D', '9H', '9S', '9C', 'AC']

In [74]:
winningprob_fivecards(a)

ValueError: ['9D', '9H', '9S', '9C', 'AC'] is not in list

In [ ]:
poker([c,d])

In [ ]:
[a,b,c,d].index(poker([c,d])
               )

In [ ]:
poker([a,b,c,d])

In [ ]:
[a,b,c,d].index(poker[a,b,c,d])

In [ ]:
[a,b,c,d].split().index(poker[a,b,c,d])

In [ ]:
[a,b,c,d].split()

In [ ]:
[a,b,c,d]

In [ ]:
whowins([a,b,c,d])

In [ ]:
e='AH AD 9C TC 5D'.split()

In [ ]:
e

In [ ]:
f='AC AS TD TH 5H'.split()

In [ ]:
f

In [ ]:
whowins([e,f])

In [ ]:
 len_handuniverse
    

In [ ]:
fulldeck

In [ ]:
len(set('4a','5a','5a'))

In [ ]:
len(set(['4a','5a','5a']))

In [ ]:
a='2a'
b='3b'
c='3c'
test=[a,b,c]

In [ ]:
test.append(['4d','5e','6f'])

In [ ]:
test

In [ ]:
a=[1,2,3,4,5]


In [ ]:
b=0
for i in a:
    b=b+1
    

In [ ]:
b

In [ ]:
x=handuniverse[3]

In [ ]:
 len_handuniverse


In [ ]:
1+1


In [295]:
fulldeck

['AS',
 'AC',
 'AD',
 'AH',
 'KS',
 'KC',
 'KD',
 'KH',
 'QS',
 'QC',
 'QD',
 'QH',
 'JS',
 'JC',
 'JD',
 'JH',
 'TS',
 'TC',
 'TD',
 'TH',
 '9S',
 '9C',
 '9D',
 '9H',
 '8S',
 '8C',
 '8D',
 '8H',
 '7S',
 '7C',
 '7D',
 '7H',
 '6S',
 '6C',
 '6D',
 '6H',
 '5S',
 '5C',
 '5D',
 '5H',
 '4S',
 '4C',
 '4D',
 '4H',
 '3S',
 '3C',
 '3D',
 '3H',
 '2S',
 '2C',
 '2D',
 '2H']

In [25]:
len(set(fulldeck))

52

In [26]:
fulldeck[0]

'2S'

In [27]:
fulldeck[51]

'AH'

In [81]:
a=1

In [29]:
b=2

In [82]:
fulldeck[a]

'AC'

In [31]:
fulldeck[b]

'2D'

In [32]:
[fulldeck[a],fulldeck[b]]

['2C', '2D']

In [289]:
handuniverse_ranked[40]

['AC', 'AD', 'AH', 'AS', 'KC']

In [343]:
twocards_0 = 2
twocards_1 = 3
sum_prob=0
i=0
for a in range (8) :
    for b in range (a+1,8) :
        for c in range (b+1,8) :
            if twocards_0 not in [a,b,c] and twocards_1 not in [a,b,c] :
                print([fulldeck[twocards_0],fulldeck[twocards_1],fulldeck[a],fulldeck[b],fulldeck[c]])

['AD', 'AH', 'AS', 'AC', 'KS']
['AD', 'AH', 'AS', 'AC', 'KC']
['AD', 'AH', 'AS', 'AC', 'KD']
['AD', 'AH', 'AS', 'AC', 'KH']
['AD', 'AH', 'AS', 'KS', 'KC']
['AD', 'AH', 'AS', 'KS', 'KD']
['AD', 'AH', 'AS', 'KS', 'KH']
['AD', 'AH', 'AS', 'KC', 'KD']
['AD', 'AH', 'AS', 'KC', 'KH']
['AD', 'AH', 'AS', 'KD', 'KH']
['AD', 'AH', 'AC', 'KS', 'KC']
['AD', 'AH', 'AC', 'KS', 'KD']
['AD', 'AH', 'AC', 'KS', 'KH']
['AD', 'AH', 'AC', 'KC', 'KD']
['AD', 'AH', 'AC', 'KC', 'KH']
['AD', 'AH', 'AC', 'KD', 'KH']
['AD', 'AH', 'KS', 'KC', 'KD']
['AD', 'AH', 'KS', 'KC', 'KH']
['AD', 'AH', 'KS', 'KD', 'KH']
['AD', 'AH', 'KC', 'KD', 'KH']
